In [ ]:
def import_corpus(path_to_file): 
    sentences = []
    sentence = []
    f = open(path_to_file)
    
    while True:
        line = f.readline()
        if not line: break  #Checks if the variable line is empty, which would indicate the end of the file. 
        #If the condition is true, it breaks out of the loop using the break statement.
            
        line = line.strip()
        line = line.lower() # i added this 
        if len(line) == 0:
            sentences.append(sentence)    
            sentence = []
            continue
                
        parts = line.split(' ')
        sentence.append((parts[0], parts[-1]))
        
    f.close()        
    return sentences 


In [ ]:
def get_one_words(corpus): # to get th list wof words with frequency=1
    freqs={}
    one_words=set()
    for sentence in corpus:
        for word in sentence:
            if word[0] in freqs:
                freqs[word[0]]+=1
            else:
                freqs[word[0]]=1

    for k,v in freqs.items():
        if v==1:
            one_words.add(k)


    return one_words

def get_states(corpus): # to get the list of states
    states=[]
    
    for sentence in corpus:
        for word in sentence:
            if word[1] not in states:
                states.append(word[1])
    
    return states 


def get_tokens(corpus): # to get the list of tokens (i.e. words)

    tokens=set()
    for sentence in corpus:
        for word in sentence:
            tokens.add(word[0])
            
    return tokens 

def change_corpus(corpus,one_words): # to add the unknown words
    for sentence in corpus:
        for i,word in enumerate(sentence):
            if word[0] in one_words :
                sentence[i] = ('unknown', word[1])

    return corpus




In [ ]:
# Exercise 1 ###################################################################
# ho cambiato leggermente la logica con cui usare queste funzioni:
# prima calcolo i parametri del modello per tutti gli stati e token con le funzioni estimate_...()
# poi con le funzioni non estimate_...() ci calcoliamo solo le prob degli stati/token che ci servono
# se ci fai caso le funzioni non estimate_...() prendono come input anche internal_representation, che è l'output delle funzioni estimate_...()
# questa è la soluzione che ho trovato per usare anche sto internal_representation di cui non sapevamo farcene

'''
Implement the probability distribution of the initial states.
Parameters:	state: string
            internal_representation: data structure representing the parameterization of this probability distribuion;
                this data structure is returned by the function estimate_initial_state_probabilities
Returns: float; initial probability of the given state
'''
def initial_state_probabilities(state, internal_representation):
    return internal_representation[state]
    
    
    
    
'''
Implement the matrix of transition probabilities.
Parameters:	from_state: string;
            to_state: string;
            internal_representation: data structure representing the parameterization of the matrix of transition probabilities;
                this data structure is returned by the function estimate_transition_probabilities
Returns: float; probability of transition from_state -> to_state
'''
def transition_probabilities(from_state, to_state, internal_representation):
    return internal_representation[f'{from_state} , {to_state}']
    
    
    
    
'''
Implement the matrix of emmision probabilities.
Parameters:	state: string;
            emission_symbol: string;
            internal_representation: data structure representing the parameterization of the matrix of emission probabilities;
                this data structure is returned by the function estimate_emission_probabilities
Returns: float; emission probability of the symbol emission_symbol if the current state is state
'''
def emission_probabilities(state, emission_symbol, internal_representation):
    return internal_representation[f'{state} , {emission_symbol}']
    
    
    
    
'''
Implement a function for estimating the parameters of the probability distribution of the initial states.
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the probability distribution of the initial states;
            use this data structure for the argument internal_representation of the function initial_state_probabilities
'''
def estimate_initial_state_probabilities(states,corpus):

    probs= {}

    for state in states:
        prob = {state:0} 
        count=0 # counts every start 

        for sentence in corpus:
            count+=1 
            if sentence[0][1]==state: # check for the first state in each sentence
                prob[state]+=1
    
        prob[state] /= count

        probs.update(prob)


    return probs

    
    
'''
Implement a function for estimating the parameters of the matrix of transition probabilities
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the matrix of transition probabilities;
            use this data structure for the argument internal_representation of the function transition_probabilities
'''
def estimate_transition_probabilities(states,corpus):
    probs={}
    for state1 in states:
        for state2 in states:
            from_freq=0 # counts how many times the from_state has a next state
            joint_freq=0 # counts how many times the from_state has as to_state as a next state 

            for sentence in corpus:
                for i,word in enumerate(sentence[:-1]): # we ingore the last word/tag since doesn't have a next state 
                    if word[1]==state1:
                        from_freq+=1
                        if sentence[i+1][1]==state2:
                            joint_freq+=1

    
            prob={f'{state1} , {state2}':joint_freq/from_freq}

            probs.update(prob)

    return probs
 
    
    
    
    
'''
Implement a function for estimating the parameters of the matrix of emission probabilities
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the matrix of emission probabilities;
            use this data structure for the argument internal_representation of the function emission_probabilities
'''
def estimate_emission_probabilities(states,tokens,corpus):
    probs={}

    for state in states:
        for token in tokens:
            state_count=0 # counts occurence of the state
            st_sy_count=0 # joint frequency of state and symbol 

            for sentence in corpus:
                for word in sentence:
                    if word[1]==state:
                        state_count+=1
                        if word[0]==token:
                            st_sy_count+=1


            prob={f'{state} , {token}':st_sy_count/state_count}

            probs.update(prob)
            
    
    return probs
    
    
    
    


In [ ]:
corpus=import_corpus('corpus_ner.txt')
one_words=get_one_words(corpus)
corpus=change_corpus(corpus,one_words)
states=get_states(corpus)
tokens=get_tokens(corpus)

In [ ]:

es_init_prob=estimate_initial_state_probabilities(states, corpus)
es_trans_prob=estimate_transition_probabilities(states, corpus)
es_emiss_prob=estimate_emission_probabilities(states,tokens, corpus)